In [ ]:
# read tsv file
import csv
import sys

import pandas as pd

In [ ]:
# pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
fileNameA = "subtask_a_train.tsv"
fileDirectoryA = "AdMIRe Subtask A Train/train"

dataA = pd.read_csv(fileDirectoryA + "/" + fileNameA, sep='\t')

In [ ]:
def sim_scores(current, sentences):
    # input = current line(example) & embeddings for sentence + captions
    
    scores = {}
    # keys = image names
    # values = scores

    embeddings = model.encode(sentences)
    similarities = model.similarity(embeddings[0], embeddings)
    # compares the embedding for the sentence including the compound 
    # with each of the embeddings, including itself and all the captions

    score1 = similarities[0][1].item()
    scores[current["image1_name"]] = score1

    score2 = similarities[0][2].item()
    scores[current["image2_name"]] = score2

    score3 = similarities[0][3].item()
    scores[current["image3_name"]] = score3

    score4 = similarities[0][4].item()
    scores[current["image4_name"]] = score4

    score5 = similarities[0][5].item()
    scores[current["image5_name"]] = score5
    
    return scores

In [ ]:
def rank_images(scores):
    ranking = []
    # scores = dictionary containing the cos similarity scores
    # comparing the sentence with the captions of the five images
    # keys = image names
    # values = scores

    for i in range(5):
        # find key which corresponds to the highest value
        m = max(scores, key=scores.get)
        # add the key (image name) to the ranking
        ranking.append(m)
        # delete the entry in the dictionary
        del scores[m]

    return ranking

In [ ]:
from evaluation_a import evaluation_single

In [ ]:
data = dataA
#data = dataA[dataA["sentence_type"]=="idiomatic"]
#data = dataA[dataA["sentence_type"]=="literal"]


total_acc = 0
total_spearman = 0

for i in range(len(data)):

    current = data.iloc[i]
    sentences = [current["sentence"], 
                 current["image1_caption"],
                 current["image2_caption"],
                 current["image3_caption"],
                 current["image4_caption"],
                 current["image5_caption"]]

    scores = sim_scores(current, sentences)
    ranking = rank_images(scores)

    # expected_order is string, not array
    exp_order = current["expected_order"].strip("['").strip("']").split("', '")
    evaluation = evaluation_single(ranking,exp_order)
    total_acc += evaluation[0]
    total_spearman += evaluation[1]

final_acc = total_acc / len(data)
print(final_acc)
final_spearman = total_spearman / len(data)
print(final_spearman)

In [ ]:
# entire subtask A training data (70):
# final_acc = 0.4
# final_spearman = 0.20142857142857135

# only literal (31):
# final_acc = 0.6774193548387096
# final_spearman = 0.3064516129032258

# only idiomatic (39):
# final_acc = 0.1794871794871795
# final_spearman = 0.11794871794871792